In [1]:
!pip install transformers==4.5.1

In [2]:
import transformers
import statistics 
from scipy import stats
import math
import pickle
import statistics
import matplotlib.pyplot as plt
import numpy as np
import torch
import pandas as pd
import seaborn as sns

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [6]:
ptweightsPath='/content/drive/My Drive/MTP/Weights/'
datasetPath='/content/drive/My Drive/MTP/Datasets/'

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
#This cell runs the statistical tests for the syntactic functional roles in the specified languages.

languages=['english']
functions=['nsubj']

model='bert-base-multilingual-cased'
MAX_LEN=64
BATCH_SIZE=8
threshold=3

bert_model = transformers.BertModel.from_pretrained(model, output_attentions=True)

for lang in languages:
  for func in functions:
    print(lang,' ',func)
    sentences_file=datasetPath + lang + '-sentences-' + func + '-1000.txt'
    tokens_file=datasetPath + lang + '-' + func + '-1000-new.txt'
    in_file=open(sentences_file,'r',encoding='utf-8')

    lines=in_file.readlines()

    tokenizer=transformers.BertTokenizer.from_pretrained(model)

    class SentenceDataset(Dataset):
      
      def __init__(self, sentences,tokenizer,max_len):
        self.sentences=sentences
        self.tokenizer=tokenizer
        self.max_len=max_len

      def __len__(self):
        return len(self.sentences)

      def __getitem__(self,item):
        sentence=str(self.sentences[item])

        encoding=tokenizer.encode_plus(
            sentence,
            max_length=self.max_len,
            add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_token_type_ids=False,
            truncation=True,
            return_tensors='pt'
        )

        return{
            'sentence':sentence,
            'input_ids':encoding['input_ids'],
            'attention_mask':encoding['attention_mask']
        }
    
    n=len(lines)

    file=open(tokens_file,"r",encoding="utf-8")

    sentences=file.read().splitlines()

    filteredLines=[]
    filteredIndices=[]
    filteredTokens=[]
    for i in range(0,len(sentences),2):
      filteredLines.append(lines[int(sentences[i].split()[0])-1])
      filteredIndices.append(sentences[i].split(' ')[1:])
      filteredTokens.append(sentences[i+1].split())

    
    def create_data_loader(sentences,tokenizer,max_len,batch_size):
      ds= SentenceDataset(
          sentences=sentences,
          tokenizer=tokenizer,
          max_len=max_len
      )

      return DataLoader(
          ds,
          batch_size=batch_size
      )


    data_loader=create_data_loader(filteredLines,tokenizer,MAX_LEN,BATCH_SIZE)

    def wordToTokenMappings(line):
  
      encoding=[tokenizer.encode_plus(
            x,
            add_special_tokens=False,
            return_token_type_ids=None
        )['input_ids'] for x in line]

      idx=1
      desired_output=[]

      for token in encoding:
        tokenoutput = []
        for ids in token:
          tokenoutput.append(idx)
          idx +=1
        desired_output.append(tokenoutput)

      return desired_output

    headSentScores=[[0 for i in range(len(filteredLines))] for j in range(145)]

    for count, data in enumerate(iter(data_loader)):
        last_hidden_state,pooler_output,attentions = bert_model(input_ids=torch.squeeze(data['input_ids'],dim=1),
                                                                  attention_mask=torch.squeeze(data['attention_mask'],dim=1),return_dict=False)
        for attention_id, attention in enumerate(attentions):
            for element in range(attention.shape[0]):
              tokens = tokenizer.tokenize(filteredLines[count * BATCH_SIZE + element])
              mappings=wordToTokenMappings(filteredTokens[count * BATCH_SIZE + element])
              indices=filteredIndices[count * BATCH_SIZE + element]
              for head in range(attention.shape[1]):
                  number = attention_id * 12 + (head + 1)
                  sentenceWeights = attention[element][head]
                  final_tokens = ['CLS'] + tokens[:62] + ['SEP']
                  sent_len = len(final_tokens)
                  array = sentenceWeights[:sent_len, :sent_len].detach().numpy()
                  sentScore=0
                  tokenCount=0
                  for index in indices:
                    dependent,head=index.split(',')
                    dependent=int(dependent)
                    head=int(head)
                    dependentTokens=mappings[dependent-1]
                    headTokens=mappings[head-1]
                    for i in dependentTokens:
                      tokenCount+=1
                      tokenScore=0
                      countHeads=0
                      for j in headTokens:
                        countHeads+=1
                        tokenScore+=array[i,j]
                      tokenScore=(tokenScore/countHeads)/(array[i,:].sum()/sent_len)
                      sentScore+=tokenScore
                  sentScore/=tokenCount
                  headSentScores[number][count * BATCH_SIZE + element]=sentScore
                    
        print(count)

    # with open(ptweightsPath + lang + '-matrix-' + func + '-1000.pl', 'wb') as f:
    #   pickle.dump(headSentScores, f)